In [301]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

# 1. load data

In [302]:
train_df = pd.read_csv("../input/train.csv")
print(train_df.describe())
print(train_df.head())
print(train_df.info())

                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726   
std       1.112799    30.202904     20.645407   181.066207   456.098091   
min       1.000

# 2. preprocessing data

In [303]:
# train_df = train_df.drop(["Alley", "PoolQC", "Fence", "MiscFeature", "Utilities", "Street", "EnclosedPorch",
#                          "ScreenPorch", "3SsnPorch", "PoolArea", "MiscVal", "SaleType", "WoodDeckSF", "OpenPorchSF",
#                          "Neighborhood", "LandSlope", "BldgType", 'LowQualFinSF'], axis=1)

In [304]:
item_label = {}
for key in train_df.keys():
    if train_df[key].dtype == object:
        item_label[key] = preprocessing.LabelEncoder().fit(train_df[key])
        train_df[key] = item_label[key].transform(train_df[key])



In [305]:
## clean data
drop_list = []
drop_list.append('Id')
for key in train_df.keys():
    tmp = train_df.loc[train_df[key] > 0, key].count()
    if tmp <1000:
        #print key, tmp
        drop_list.append(key)
    #print key, tmp
train_df = train_df.drop(drop_list, axis=1)


In [306]:
def FillNan(df):
    item = []
    for key in df.keys():
        if df.loc[df[key].isnull(), key].shape[0] != 0:
            item.append(key)
    
    for key in item:
        df_tmp = df.reindex_axis([key] + list([a for a in df.columns if a not in item]), axis=1)
    
        unkown_data = df_tmp.loc[df_tmp[key].isnull()]
    
        known_data = df_tmp.loc[df_tmp[key].notnull()]
        
        rfr = RandomForestRegressor(n_estimators=400, n_jobs=-1)
        
        rfr.fit(known_data.values[:, 1::], known_data.values[:, 0])
        predicts = rfr.predict(unkown_data.values[:, 1::])
        
        df.loc[(df[key].isnull()), key] = predicts
        print ("key:%s, known_data:%s" % (key, predicts))
    return df
#print train_df.columns.get_loc("MSSubClass")
FillNan(train_df)

key:LotFrontage, known_data:[  92.0825   78.84     89.745    59.1575   66.0025   71.0375   67.5075
   59.945    96.04     74.91     98.3425   68.895    67.6      79.1675
   76.96     61.595    59.81    150.09     78.6725   80.6075   46.1625
   94.045    60.26     86.1175   67.9525   52.7925   60.63    103.78
   59.09     76.8425  122.175    78.3725   84.7975   24.575    73.53
   66.9825   37.2125   78.055    79.1425   68.7725  104.7975   63.8
   64.8575   70.2025   71.005    86.9275   72.4175   66.1825   71.955
  123.0775   61.115    75.155    64.545    76.7875   77.875    75.145
  132.8575   62.84     54.435    91.6775   44.4525   74.115    62.73
   61.9975   73.7075   81.03     78.25     62.21     73.03    126.0175
   69.32     59.455    71.895    84.97     51.67     81.385    79.57
   93.0125   78.36     96.7075   53.4425   62.57     44.1325   49.165
   67.12     34.3725  101.055    94.6275   62.445    99.3975   67.965
   78.3175   79.9175   78.84    101.8825   44.9      79.4975   7

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LandContour,LotConfig,Neighborhood,Condition1,Condition2,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,3,65.0000,8450,1,3,4,5,2,2,...,2,548,5,5,2,2,2008,8,4,208500
1,20,3,80.0000,9600,1,3,2,24,1,2,...,2,460,5,5,2,5,2007,8,4,181500
2,60,3,68.0000,11250,1,3,4,5,2,2,...,2,608,5,5,2,9,2008,8,4,223500
3,70,3,60.0000,9550,1,3,0,6,2,2,...,3,642,5,5,2,2,2006,8,0,140000
4,60,3,84.0000,14260,1,3,2,15,2,2,...,3,836,5,5,2,12,2008,8,4,250000
5,50,3,85.0000,14115,1,3,4,11,2,2,...,2,480,5,5,2,10,2009,8,4,143000
6,20,3,75.0000,10084,1,3,4,21,2,2,...,2,636,5,5,2,8,2007,8,4,307000
7,60,3,92.0825,10382,1,3,0,14,4,2,...,2,484,5,5,2,11,2009,8,4,200000
8,50,4,51.0000,6120,1,3,4,17,0,2,...,2,468,2,5,2,4,2008,8,0,129900
9,190,3,50.0000,7420,1,3,0,3,0,0,...,1,205,3,5,2,1,2008,8,4,118000


### standrd scaler

In [307]:
train_X, test_X, train_y, test_y = train_test_split(train_df.values[:, :-1], train_df.values[:, -1], random_state=2016, test_size=.3)

In [313]:
from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


In [314]:
lr = LinearRegression()
lr.fit(train_X, train_y)
predict = lr.predict(test_X)
print rmse(test_y, predict)
#print train_X.shape

36227.051554


In [309]:
from sklearn.preprocessing import StandardScaler
train_df.values[:, :-1] = StandardScaler().fit_transform(train_df.values[:, :-1])

train_X, test_X, train_y, test_y = train_test_split(train_df.values[:, :-1], train_df.values[:, -1], random_state=2016, test_size=.3)
lr = LinearRegression()
lr.fit(train_X, train_y)
lr.score(test_X, test_y)
#print train_X.shape

0.81719361005529556

In [310]:
from sklearn.linear_model import LassoCV 

lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.005]).fit(train_df.values[:, :-1],train_df.values[:, -1]) 


# Predict test

In [311]:
def predict_test(model):
    test_df = pd.read_csv("../input/test.csv")
    from sklearn.preprocessing import Imputer
    id_tmp = test_df["Id"]
    test_df = test_df.drop(drop_list, axis=1)
    
    #print test_df.MSZoning
    
    # fill nan 
    # test_df = test_df.fillna(-1)

    for key in test_df.keys():
        #print (test_df[key].mode()).values
        test_df.loc[(test_df[key].isnull()), key] = (test_df[key].mode()).values
    #print test_df
            
    for key in test_df.keys():
        if test_df[key].dtype == object:
            #print key, item_label.keys()
            if key in item_label:
                test_df[key] = item_label[key].transform(test_df[key])
    predict = model.predict(test_df.values)
    print predict
    submission = pd.DataFrame({
            "Id": id_tmp,
            "SalePrice": predict
        })
    submission.to_csv("result.csv", index=False)


predict_test(lasso)

[ 107192.85521459  166783.190294    171207.47688219 ...,  141044.92952159
  105410.58177286  252309.61274994]


In [312]:
a ={'a':1, 'b':2}
for i in a:
    print i

a
b
